
Bitcoin uses ECC([Elliptic curve cryptography](https://en.wikipedia.org/wiki/Elliptic-curve_cryptography)) to secure the transactions, which is a kind of [public-key cryptography](https://en.wikipedia.org/wiki/Public-key_cryptography). Compared to the more widely used RSA, ECC has shorter keys and is faster than RSA when generating keys; however it appearred later than RSA and many ECC algorithms are protected by patents, therefore not as popular as RSA.

In general, public-key crypotography algorithms works in this way: generate a pair of integers (public key, private key), where it is computationally easy to calculate public key whereas impossible the other way. The keys are used to transform data - for example represent your data as a number, then calculate new data = old data * pulbic key. Let's say the keys do the following operations
- pulic key: data -> F(data)
- private key: data -> G(data)

so that F(G(data)) = data, G(F(data)) = data for any data. The following examples are two usages of the keys:

1. Encrypting & decrypting data: When A has the public key and some data, B has the private key. A wants to send the data to B, so A transform the data with public key and send F(data) to B. Then, B recovers the data by doing G(F(data)). Anybody that has F(data) and public key cannot recover because they don't have the public key and it's computationally hard to figure out the public key. 
2. Signature: When B has private key and want to request a transaction (for example send money to C), then he could send out G(transaction request). Other people can recover the transaction request by doing F(G(transaction request)), while knowing that the sender must be B but not other people, because it's impossible to G(transaction request) without the private key.






The central concept in ECC is the elliptic curve, which is a set of points that satisfy a mathematical equation. In mathematical terms, an elliptic curve is 
$$
    C = \{(x, y)| y^2 = x^3 + ax^2 + b \text{mod} p, 4a^3+27b^2 \neq 0 \} \cup \{ 0 \}
$$
A elliptic curve is uniquely defined by three numbers: a prime number $p$ and two integers $a$ and $b$. 


First, we would like to implement some methematical concepts in ellipitc curve with python classes.

If you have no programming experience with python, or with OOP (Object-Oriented-Programming) in general, a `class` can be understood as a cateogry of things, like "animal", "dog", etc; while an `object` is an instance of an `class`, for example *the* cat you came across on the street yesterday is an object of the "cat" class. [A friendly introduction to OOP in python](https://realpython.com/python3-object-oriented-programming/)

Python has a vanilla way to define classes, like below:
```python
class cat:
    age: int
    def eat(self):
        ...
```
but we would like some nice features automatically constructed (which is called decorating) when we define the classes. Therefore, we import the `dataclass` decorator from the `dataclasses` python library. A class decorated by `dataclass` automatically have certain methods defined in a reasonable way. The way to use the `dataclass` decorator is adding a line before the definition of the class, like below:
```python
@dataclass
class cat:
    ...
``` 
 For more details about `dataclass`, see the [official documentation](https://docs.python.org/3/library/dataclasses.html).

Now we are ready to define the elliptic curve class:

In [51]:
from dataclasses import dataclass 

@dataclass
class EllipticCurve:
    """
    Elliptic curve over a filed F_p.
    More specifically, an elliptic curve is the set of points defined by 
    y^2 = x^2 + ax + b
    """
    p: int 
    a: int
    b: int

# an simple example, using 7 as the prime number and a=2, b=3:
my_curve = EllipticCurve(7, 2, 3)
print(my_curve)

EllipticCurve(p=7, a=2, b=3)


Now e define the elliptic curve [secp256k1](https://en.bitcoin.it/wiki/Secp256k1) used by bitcoin. It is an elliptic curve with p = `FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFE FFFFFC2F` (base 16) which equals $$2^{256} - 2^{32} - 2^9 - 2^8 - 2^7 - 2^6 - 2^4 - 1$$
with a=0 and b=7.

In [52]:
secp256k1 = EllipticCurve(
    p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F, # the 0x in the beginning indicates base 16
    a = 0x0000000000000000000000000000000000000000000000000000000000000000, # a = 0
    b = 0x0000000000000000000000000000000000000000000000000000000000000007, # b = 7
)
secp256k1

EllipticCurve(p=115792089237316195423570985008687907853269984665640564039457584007908834671663, a=0, b=7)

Now we define the points on the elliptic curve, which are simply integer pairs (x, y), for a given elliptic curve.

In [53]:
@dataclass
class Point:
    """
    A point on an elliptic curve.
    """
    curve: EllipticCurve
    x: int
    y: int

    def is_valid(self):
        """ Check the Point is on the curve or not.
        """
        if ( self.y * self.y - self.x ** 3 - self.curve.a * self.x - self.curve.b ) % self.curve.p == 0:
            print(f"({self.x}, {self.y}) is on {self.curve}")
            return True
        else:
            return False

No we define a generator which a point on the curve together with a pre-chosen order n.

In [54]:
@dataclass
class Generator:

    g: Point
    n: int # order of the generator

G = Point(
    secp256k1,
    x = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798,
    y = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8,
)
print(G.is_valid()) # check validity of point

secret_key = int.from_bytes(b'May the force be with your', 'big')
print(secret_key)

(55066263022277343669578718895168534326250603453777594175500187360389116729240, 32670510020758816978083085130507043184471273380659243275938904335757337482424) is on EllipticCurve(p=115792089237316195423570985008687907853269984665640564039457584007908834671663, a=0, b=7)
True
124346078296186283466506634239948744353101870944869081959855474
